<a href="https://colab.research.google.com/github/MarshaGomez/F5-MOEA-Variations/blob/main/examples/Genetic_Algorithm_from_Scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
from collections import namedtuple
from typing import List, Callable, Tuple
from random import choices, randint, randrange, random
from functools import partial

import time

# The genome is a list of ones and zeros
Genome = List[int]

# The population is a list of Genomes
Population = List[Genome]

# Fitness Function to take a genome and return a fitness value to make the correct choices
FitnessFunc = Callable[[Genome], int]

PopulateFunc = Callable[[], Population]

SelectionFunc = Callable[[Population, FitnessFunc], Tuple[Genome, Genome]]

CrossoverFunc = Callable[[Genome, Genome], Tuple[Genome, Genome]]

MutationFunc = Callable[[Genome], Genome]
# Structure with a different list of things
Thing = namedtuple('Thing', ['name', 'value', 'weight'])


# Different list of things
things = [
          Thing('Laptop', 500, 2200),
          Thing('Headphones', 150, 160),
          Thing('Coffe Mug', 60, 350),
          Thing('Notepad', 40, 333),
          Thing('Water Bottle', 30, 192),
]


more_things = [
          Thing('Mints', 5, 25),
          Thing('Socks', 10, 38),
          Thing('Tissues', 15, 80),
          Thing('Phone', 500, 200),
          Thing('Baseball Cap', 100, 70),
] + things


# Genetic Representating of a solution
def generate_genome(length: int) -> Genome:
  return choices([0,1], k=length)

# Function to generate new solutions
def generate_population(size: int, genome_length: int) -> Population:
  return [generate_genome(genome_length) for _ in range(size)]

# Fitness function to evaluate solutions
def fitness(genome: Genome, things: [Thing], weight_limit: int) -> int:
  if len(genome) != len(things):
    raise ValueError("Genome and things must be of the same length")
  
  weight = 0
  value = 0

  for i, thing in enumerate(things):
    if genome[i] == 1:
      weight += thing.weight
      value += thing.value

      # Turn zero because this solution is invalid
      if weight > weight_limit:
        return 0

  return value


# Select function to select the solutions to generate the next generation
def selection_pair(population: Population, fitness_func: FitnessFunc) -> Population:
  return choices(
      population = population,
      weights = [fitness_func(genome) for genome in population],
      k = 2
  )

# Crossover function. Single point takes two genomes as parameters and returns two genomes as outputs
def single_point_crossover(a:Genome, b:Genome) -> Tuple[Genome, Genome]:
  # Check both genomes with the same length
  if len(a) != len(b):
    raise ValueError("Genomes a and b must be of same length")
  
  length = len(a)
  if length < 2:
    return a, b


  # Random index
  p = randint(1, length -1)
  return a[0:p] + b[p:], b[0:p] + a[p:]


# Mutation function
def mutation(genome: Genome, num: int = 1, probability: float = 0.5) -> Genome:
  for _ in range(num):
    # Random index
    index = randrange(len(genome))

    # abs + absolute value 
    genome[index] = genome[index] if random() > probability else abs(genome[index] -1)

  return genome


In [24]:
# Function to run the evolution
def run_evolution(
    populate_func: PopulateFunc,
    fitness_func: FitnessFunc,
    fitness_limit: int,
    selection_func: SelectionFunc = selection_pair,
    crossover_func: CrossoverFunc = single_point_crossover,
    mutation_func: MutationFunc = mutation,
    generation_limit: int = 100
) -> Tuple[Population, int]:

  population = populate_func()

  for i in range(generation_limit):
    population = sorted(
        population,
        key = lambda genome: fitness_func(genome),
        reverse = True
    )

    if fitness_func(population[0]) >= fitness_limit:
      break
    next_generation = population[0:2]

    for j in range(int(len(population) / 2 ) -1):
      parents = selection_func(population, fitness_func)
      offspring_a, offspring_b = crossover_func(parents[0], parents[1])
      offspring_a = mutation_func(offspring_a)
      offspring_b = mutation_func(offspring_b)
      next_generation += [offspring_a, offspring_b]

    population = next_generation
  
  population = sorted(
        population,
        key = lambda genome: fitness_func(genome),
        reverse = True
    )
  
  return population, i

start = time.time()
population, generations = run_evolution(
    populate_func = partial(
        generate_population, size=10, genome_length=len(things)
    ),
    fitness_func=partial(
        fitness, things=things, weight_limit=3000
    ),
    fitness_limit=748,
    generation_limit=100
)
end = time.time()

def genome_to_things(genome: Genome, things: [Thing]) -> [Thing]:
  result = []
  for i, thing, in enumerate(things):
    if genome[i] == 1:
      result += [thing.name]

  return result

print(f"number of generation: {generations}")
print(f"time of generation: {end - start}s")
print(f"best solution: {genome_to_things(population[0], things)}")

number of generation: 99
time of generation: 0.017794370651245117s
best solution: ['Laptop', 'Headphones', 'Coffe Mug', 'Water Bottle']
